#### Name: Ritu Pal
#### USN: 1AUA23BCS906

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import random
from faker import Faker
from datetime import datetime

# Initialize Spark
spark = SparkSession.builder \
    .appName("Employee Job Recommendation System") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext
fake = Faker()

# Q1. Employee Data Simulation & RDD Operations

## You are asked to simulate a dataset of 1000 employees with the following attributes:

* employee_id (unique ID)
* user_name
* city
* age
* ctc (in LPA)
* experience_years
* skillset
* company_name
* linkedin_profile

### Task 1: Generate dataset and create RDD in PySpark

In [4]:
print("="*80)
print("Q1: EMPLOYEE DATA SIMULATION & RDD OPERATIONS")
print("="*80)

# Define data pools
cities = ["Ahmedabad", "Mumbai", "Bangalore", "Delhi", "Pune", "Hyderabad", "Chennai", "Kolkata"]
skillsets_pool = [
    "Machine Learning", "Data Science", "Python", "Java", "React",
    "Node.js", "AWS", "Docker", "Kubernetes", "SQL", "MongoDB",
    "Deep Learning", "NLP", "Computer Vision", "DevOps", "Angular"
]
companies = ["TCS", "Infosys", "Wipro", "Google", "Amazon", "Microsoft",
              "Accenture", "Cognizant", "HCL", "Tech Mahindra"]

def generate_employee(emp_id):
    """Generate a single employee record"""
    name = fake.name()
    city = random.choice(cities)
    age = random.randint(22, 55)
    ctc = round(random.uniform(3.0, 30.0), 2)
    exp = random.randint(0, 20)
    # Each employee has 2-5 skills
    num_skills = random.randint(2, 5)
    skills = random.sample(skillsets_pool, num_skills)
    company = random.choice(companies)
    linkedin = f"https://linkedin.com/in/{name.lower().replace(' ', '-')}-{emp_id}"

    return {
        'employee_id': emp_id,
        'user_name': name,
        'city': city,
        'age': age,
        'ctc': ctc,
        'experience_years': exp,
        'skillset': skills,
        'company_name': company,
        'linkedin_profile': linkedin
    }

# Generate 1000 employees
employee_data = [generate_employee(i) for i in range(1, 1001)]

# Create RDD
employee_rdd = sc.parallelize(employee_data)

print(f"\nTotal Employees Generated: {employee_rdd.count()}")
print("\nSample Employee Records:")
for emp in employee_rdd.take(3):
    print(f"  ID: {emp['employee_id']}, Name: {emp['user_name']}, "
          f"City: {emp['city']}, CTC: {emp['ctc']} LPA, "
          f"Exp: {emp['experience_years']} yrs, Skills: {emp['skillset'][:2]}")

Q1: EMPLOYEE DATA SIMULATION & RDD OPERATIONS

Total Employees Generated: 1000

Sample Employee Records:
  ID: 1, Name: Jose Ramirez, City: Pune, CTC: 14.75 LPA, Exp: 6 yrs, Skills: ['Python', 'Node.js']
  ID: 2, Name: Andrea Edwards, City: Pune, CTC: 20.44 LPA, Exp: 5 yrs, Skills: ['AWS', 'Deep Learning']
  ID: 3, Name: Greg Hines, City: Pune, CTC: 9.12 LPA, Exp: 1 yrs, Skills: ['Angular', 'Deep Learning']


### Task 2: Divide RDD into 3 clusters (e.g., based on city, skillset, or experience)

In [5]:
print("\n" + "-"*80)
print("Task 2: Dividing Employees into 3 Clusters")
print("-"*80)

# Cluster by experience level
def assign_cluster(emp):
    exp = emp['experience_years']
    if exp <= 3:
        return ('Junior', emp)
    elif exp <= 10:
        return ('Mid-Level', emp)
    else:
        return ('Senior', emp)

clustered_rdd = employee_rdd.map(assign_cluster)

# Count employees in each cluster
cluster_counts = clustered_rdd.countByKey()
print("\nCluster Distribution:")
for cluster, count in sorted(cluster_counts.items()):
    print(f"  {cluster}: {count} employees")


--------------------------------------------------------------------------------
Task 2: Dividing Employees into 3 Clusters
--------------------------------------------------------------------------------

Cluster Distribution:
  Junior: 181 employees
  Mid-Level: 340 employees
  Senior: 479 employees


### Task 3: Apply filter transformation
* Experience > 3 years
* CTC < 15 LPA
* City = Ahmedabad
* Skillset contains Machine Learning

In [6]:
print("\n" + "-"*80)
print("Task 3: Filtered Employees (Exp > 3, CTC < 15, City=Ahmedabad, ML skill)")
print("-"*80)

filtered_employees = employee_rdd.filter(
    lambda emp: emp['experience_years'] > 3 and
                emp['ctc'] < 15 and
                emp['city'] == 'Ahmedabad' and
                'Machine Learning' in emp['skillset']
)

filtered_count = filtered_employees.count()
print(f"\nFiltered Employees Count: {filtered_count}")
if filtered_count > 0:
    print("\nSample Filtered Employees:")
    for emp in filtered_employees.take(5):
        print(f"  {emp['user_name']} - CTC: {emp['ctc']} LPA, "
              f"Exp: {emp['experience_years']} yrs, Skills: {emp['skillset']}")


--------------------------------------------------------------------------------
Task 3: Filtered Employees (Exp > 3, CTC < 15, City=Ahmedabad, ML skill)
--------------------------------------------------------------------------------

Filtered Employees Count: 12

Sample Filtered Employees:
  Sharon Baker - CTC: 6.57 LPA, Exp: 16 yrs, Skills: ['Kubernetes', 'Deep Learning', 'Machine Learning']
  Amy Gonzalez - CTC: 10.54 LPA, Exp: 4 yrs, Skills: ['React', 'Machine Learning', 'MongoDB', 'Computer Vision', 'Angular']
  Kimberly Richardson - CTC: 11.91 LPA, Exp: 8 yrs, Skills: ['Machine Learning', 'React', 'Docker', 'DevOps']
  Amy Harris - CTC: 4.06 LPA, Exp: 10 yrs, Skills: ['Node.js', 'Machine Learning', 'Python', 'Java']
  Melissa Stanley - CTC: 7.26 LPA, Exp: 20 yrs, Skills: ['Java', 'Computer Vision', 'Machine Learning', 'AWS']


# Q2: Job Posting Data & RDD Analysis
## Simulate a dataset of 5000 job postings with the following attributes:

* job_id
* job_role
* job_skillset
* experience_required (in years)
* ctc_expected (in LPA)
* city
* company_name

### Task 1: Create RDD from this dataset

In [7]:
print("\n" + "="*80)
print("Q2: JOB POSTING DATA & RDD ANALYSIS")
print("="*80)

job_roles = [
    "Data Scientist", "ML Engineer", "Software Engineer", "DevOps Engineer",
    "Full Stack Developer", "Backend Developer", "Frontend Developer",
    "Data Engineer", "AI Researcher", "Cloud Architect", "Product Manager"
]

def generate_job(job_id):
    """Generate a single job posting"""
    role = random.choice(job_roles)
    num_skills = random.randint(2, 4)
    skills = random.sample(skillsets_pool, num_skills)
    exp_required = random.randint(0, 15)
    ctc_expected = round(random.uniform(5.0, 35.0), 2)
    city = random.choice(cities)
    company = random.choice(companies)

    return {
        'job_id': job_id,
        'job_role': role,
        'job_skillset': skills,
        'experience_required': exp_required,
        'ctc_expected': ctc_expected,
        'city': city,
        'company_name': company
    }

# Generate 5000 jobs
job_data = [generate_job(i) for i in range(1, 5001)]

# Create RDD
job_rdd = sc.parallelize(job_data)

print(f"\nTotal Job Postings Generated: {job_rdd.count()}")
print("\nSample Job Postings:")
for job in job_rdd.take(3):
    print(f"  ID: {job['job_id']}, Role: {job['job_role']}, "
          f"City: {job['city']}, CTC: {job['ctc_expected']} LPA, "
          f"Exp Required: {job['experience_required']} yrs")


Q2: JOB POSTING DATA & RDD ANALYSIS

Total Job Postings Generated: 5000

Sample Job Postings:
  ID: 1, Role: ML Engineer, City: Mumbai, CTC: 29.34 LPA, Exp Required: 7 yrs
  ID: 2, Role: Cloud Architect, City: Kolkata, CTC: 13.75 LPA, Exp Required: 3 yrs
  ID: 3, Role: ML Engineer, City: Ahmedabad, CTC: 28.35 LPA, Exp Required: 9 yrs


### Task 2a: Find all jobs in Ahmedabad

In [8]:
print("\n" + "-"*80)
print("Task 2a: Jobs in Ahmedabad")
print("-"*80)

ahmedabad_jobs = job_rdd.filter(lambda job: job['city'] == 'Ahmedabad')
ahmedabad_count = ahmedabad_jobs.count()
print(f"\nJobs in Ahmedabad: {ahmedabad_count}")
print("\nSample Jobs:")
for job in ahmedabad_jobs.take(3):
    print(f"  {job['job_role']} at {job['company_name']} - "
          f"CTC: {job['ctc_expected']} LPA, Exp: {job['experience_required']} yrs")


--------------------------------------------------------------------------------
Task 2a: Jobs in Ahmedabad
--------------------------------------------------------------------------------

Jobs in Ahmedabad: 616

Sample Jobs:
  ML Engineer at Google - CTC: 28.35 LPA, Exp: 9 yrs
  Frontend Developer at Wipro - CTC: 26.59 LPA, Exp: 10 yrs
  ML Engineer at Accenture - CTC: 11.33 LPA, Exp: 9 yrs


### Task 2b: Filter jobs (exp > 2, CTC < 20)

In [9]:
print("\n" + "-"*80)
print("Task 2b: Jobs with Exp > 2 years and CTC < 20 LPA")
print("-"*80)

filtered_jobs = job_rdd.filter(
    lambda job: job['experience_required'] > 2 and job['ctc_expected'] < 20
)
filtered_jobs_count = filtered_jobs.count()
print(f"\nFiltered Jobs Count: {filtered_jobs_count}")
print("\nSample Jobs:")
for job in filtered_jobs.take(3):
    print(f"  {job['job_role']} - CTC: {job['ctc_expected']} LPA, "
          f"Exp Required: {job['experience_required']} yrs, City: {job['city']}")


--------------------------------------------------------------------------------
Task 2b: Jobs with Exp > 2 years and CTC < 20 LPA
--------------------------------------------------------------------------------

Filtered Jobs Count: 2096

Sample Jobs:
  Cloud Architect - CTC: 13.75 LPA, Exp Required: 3 yrs, City: Kolkata
  Software Engineer - CTC: 16.41 LPA, Exp Required: 6 yrs, City: Delhi
  AI Researcher - CTC: 12.42 LPA, Exp Required: 11 yrs, City: Mumbai


### Task 2c: Count jobs by city

In [10]:
print("\n" + "-"*80)
print("Task 2c: Job Postings Count by City")
print("-"*80)

jobs_by_city = job_rdd.map(lambda job: (job['city'], 1)).reduceByKey(lambda a, b: a + b)
print("\nJobs per City:")
for city, count in sorted(jobs_by_city.collect(), key=lambda x: x[1], reverse=True):
    print(f"  {city}: {count} jobs")


--------------------------------------------------------------------------------
Task 2c: Job Postings Count by City
--------------------------------------------------------------------------------

Jobs per City:
  Chennai: 659 jobs
  Kolkata: 637 jobs
  Pune: 631 jobs
  Delhi: 630 jobs
  Mumbai: 629 jobs
  Ahmedabad: 616 jobs
  Hyderabad: 602 jobs
  Bangalore: 596 jobs


# Q3: Job Recommendation System Using RDD Joins
## Using the employee dataset (Q1) and job dataset (Q2), design a recommendation mechanism.

In [13]:
def calculate_skillset_match(emp_skills, job_skills):
    """Calculate percentage of job skills matched by employee"""
    if not job_skills:
        return 0.0
    matched = len(set(emp_skills) & set(job_skills))
    return matched / len(job_skills)

def calculate_recommendation_score(emp, job):
    """Calculate weighted recommendation score"""
    # Weight priorities as specified in Task 3
    SKILLSET_WEIGHT = 0.4    # Highest weight
    EXPERIENCE_WEIGHT = 0.3  # Second highest weight
    CTC_WEIGHT = 0.2         # Third highest weight
    CITY_WEIGHT = 0.1        # Fourth highest weight

    # Skillset match score (0 to 1)
    skillset_score = calculate_skillset_match(emp['skillset'], job['job_skillset'])

    # Experience fit score (1 if qualified, 0.5 if overqualified, 0 if underqualified)
    if emp['experience_years'] >= job['experience_required']:
        exp_diff = emp['experience_years'] - job['experience_required']
        # Penalize being too overqualified
        experience_score = max(0.5, 1.0 - (exp_diff * 0.05))
    else:
        experience_score = 0.0

    # CTC fit score (1 if employee CTC <= job CTC, scaled down if not)
    if emp['ctc'] <= job['ctc_expected']:
        ctc_score = 1.0
    else:
        # Penalize if employee expects more than job offers
        ctc_diff = emp['ctc'] - job['ctc_expected']
        ctc_score = max(0.0, 1.0 - (ctc_diff / 10.0))

    # City match score (1 if same city, 0 otherwise)
    city_score = 1.0 if emp['city'] == job['city'] else 0.0

    # Calculate weighted score
    total_score = (
        skillset_score * SKILLSET_WEIGHT +
        experience_score * EXPERIENCE_WEIGHT +
        ctc_score * CTC_WEIGHT +
        city_score * CITY_WEIGHT
    )

    return total_score

### Task 1: Map employee skillsets with job skill requirements

In [14]:
print("\nTask 1: Mapping employee skillsets with job skill requirements...")

def generate_recommendations(emp):
    """Generate recommendations for a single employee"""
    emp_id = emp['employee_id']
    recommendations = []

    # Task 1 & 2: Map skillsets and filter jobs where skillset matches
    for job in job_data:
        # Task 2: Only recommend if skillset matches (at least one skill in common)
        if set(emp['skillset']) & set(job['job_skillset']):
            # Task 3: Calculate ranking weight based on multiple factors
            score = calculate_recommendation_score(emp, job)
            if score > 0:
                recommendations.append((score, job))

    # Task 4: Sort by weight and take top 3 recommendations
    recommendations.sort(key=lambda x: x[0], reverse=True)
    top_3 = recommendations[:3]

    return (emp_id, emp, top_3)

# Generate recommendations for all employees using RDD map operation
recommendations_rdd = employee_rdd.map(generate_recommendations)

print("✓ Task 1 Complete: Employee skillsets mapped with job requirements")


Task 1: Mapping employee skillsets with job skill requirements...
✓ Task 1 Complete: Employee skillsets mapped with job requirements


### Task 2: Recommend jobs only if skillset matches

In [15]:
print("\nTask 2: Jobs recommended only when skillset matches")
print("         (Filtering applied - no recommendations without skill overlap)")


Task 2: Jobs recommended only when skillset matches
         (Filtering applied - no recommendations without skill overlap)


### Task 3: Ranking weights assigned based on multiple factors

In [16]:
print("\nTask 3: Ranking weights assigned:")
print("         • Skillset match: 40% (highest weight)")
print("         • Experience fit: 30% (second highest weight)")
print("         • CTC expectation: 20% (third highest weight)")
print("         • City match: 10% (fourth highest weight)")


Task 3: Ranking weights assigned:
         • Skillset match: 40% (highest weight)
         • Experience fit: 30% (second highest weight)
         • CTC expectation: 20% (third highest weight)
         • City match: 10% (fourth highest weight)


### Task 4: Output top 3 recommended jobs for each employee

In [17]:
print("\n" + "-"*80)
print("Task 4: Top 3 Recommended Jobs for Sample Employees (Ranked by Weight)")
print("-"*80)

sample_recommendations = recommendations_rdd.take(5)

for emp_id, emp, recs in sample_recommendations:
    print(f"\n{'='*70}")
    print(f"Employee: {emp['user_name']} (ID: {emp_id})")
    print(f"Current: {emp['city']}, CTC: {emp['ctc']} LPA, "
          f"Exp: {emp['experience_years']} yrs")
    print(f"Skills: {', '.join(emp['skillset'][:3])}")
    print(f"\nTop 3 Recommended Jobs:")

    for rank, (score, job) in enumerate(recs, 1):
        skill_match = calculate_skillset_match(emp['skillset'], job['job_skillset'])
        print(f"\n  Rank {rank} - Score: {score:.3f}")
        print(f"    Job: {job['job_role']} at {job['company_name']}")
        print(f"    Location: {job['city']}, CTC: {job['ctc_expected']} LPA")
        print(f"    Exp Required: {job['experience_required']} yrs")
        print(f"    Required Skills: {', '.join(job['job_skillset'])}")
        print(f"    Skill Match: {skill_match*100:.1f}%")

# Statistics
print("\n" + "="*80)
print("RECOMMENDATION STATISTICS")
print("="*80)

total_recommendations = recommendations_rdd.map(
    lambda x: len(x[2])
).reduce(lambda a, b: a + b)

employees_with_recs = recommendations_rdd.filter(
    lambda x: len(x[2]) > 0
).count()

avg_recs = total_recommendations / employee_rdd.count()

print(f"\nTotal Employees: {employee_rdd.count()}")
print(f"Employees with Recommendations: {employees_with_recs}")
print(f"Average Recommendations per Employee: {avg_recs:.2f}")

# Find employees with highest scoring recommendations
print("\n" + "-"*80)
print("Employees with Best Job Matches")
print("-"*80)

best_matches = recommendations_rdd.filter(
    lambda x: len(x[2]) > 0
).map(
    lambda x: (x[0], x[1], x[2][0][0])  # emp_id, emp, best_score
).sortBy(
    lambda x: x[2], ascending=False
).take(5)

for emp_id, emp, best_score in best_matches:
    print(f"\n  {emp['user_name']} - Best Match Score: {best_score:.3f}")
    print(f"    Skills: {', '.join(emp['skillset'][:3])}")
    print(f"    Experience: {emp['experience_years']} yrs, "
          f"CTC: {emp['ctc']} LPA, City: {emp['city']}")

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)

# Generate recommendations for all employees
recommendations_rdd = employee_rdd.map(generate_recommendations)

print("✓ Recommendations generated for all employees using RDD map operation")


--------------------------------------------------------------------------------
Task 4: Top 3 Recommended Jobs for Sample Employees (Ranked by Weight)
--------------------------------------------------------------------------------

Employee: Jose Ramirez (ID: 1)
Current: Pune, CTC: 14.75 LPA, Exp: 6 yrs
Skills: Python, Node.js, React

Top 3 Recommended Jobs:

  Rank 1 - Score: 1.000
    Job: DevOps Engineer at Cognizant
    Location: Pune, CTC: 16.25 LPA
    Exp Required: 6 yrs
    Required Skills: Angular, React
    Skill Match: 100.0%

  Rank 2 - Score: 1.000
    Job: Software Engineer at Tech Mahindra
    Location: Pune, CTC: 25.85 LPA
    Exp Required: 6 yrs
    Required Skills: Python, Angular
    Skill Match: 100.0%

  Rank 3 - Score: 0.985
    Job: DevOps Engineer at HCL
    Location: Pune, CTC: 18.21 LPA
    Exp Required: 5 yrs
    Required Skills: React, Node.js
    Skill Match: 100.0%

Employee: Andrea Edwards (ID: 2)
Current: Pune, CTC: 20.44 LPA, Exp: 5 yrs
Skills: AWS, 

# Recommendation System Statistics

In [18]:
print("\n" + "="*80)
print("RECOMMENDATION SYSTEM STATISTICS")
print("="*80)

total_recommendations = recommendations_rdd.map(
    lambda x: len(x[2])
).reduce(lambda a, b: a + b)

employees_with_recs = recommendations_rdd.filter(
    lambda x: len(x[2]) > 0
).count()

employees_without_recs = recommendations_rdd.filter(
    lambda x: len(x[2]) == 0
).count()

avg_recs = total_recommendations / employee_rdd.count()

print(f"\nOVERALL METRICS:")
print(f"  Total Employees Analyzed: {employee_rdd.count()}")
print(f"  Employees with Recommendations: {employees_with_recs} ({employees_with_recs/employee_rdd.count()*100:.1f}%)")
print(f"  Employees without Recommendations: {employees_without_recs} ({employees_without_recs/employee_rdd.count()*100:.1f}%)")
print(f"  Total Recommendations Generated: {total_recommendations}")
print(f"  Average Recommendations per Employee: {avg_recs:.2f}")
print(f"  Total Job Postings Available: {job_rdd.count()}")

# Distribution of recommendation counts
rec_distribution = recommendations_rdd.map(
    lambda x: (len(x[2]), 1)
).reduceByKey(lambda a, b: a + b).collect()

print(f"\nRECOMMENDATION DISTRIBUTION:")
for num_recs, count in sorted(rec_distribution):
    print(f"  {num_recs} recommendations: {count} employees ({count/employee_rdd.count()*100:.1f}%)")

# Find employees with highest scoring recommendations
print("\n" + "-"*80)
print("TOP 5 EMPLOYEES WITH BEST JOB MATCHES")
print("-"*80)
print("(Employees whose top recommendation has the highest score)\n")

best_matches = recommendations_rdd.filter(
    lambda x: len(x[2]) > 0
).map(
    lambda x: (x[0], x[1], x[2][0][0], x[2][0][1])  # emp_id, emp, best_score, best_job
).sortBy(
    lambda x: x[2], ascending=False
).take(5)

for rank, (emp_id, emp, best_score, best_job) in enumerate(best_matches, 1):
    print(f"{rank}. {emp['user_name']} - Match Score: {best_score:.3f}")
    print(f"   Employee: {', '.join(emp['skillset'][:3])}, {emp['experience_years']} yrs exp, {emp['city']}")
    print(f"   Best Match: {best_job['job_role']} at {best_job['company_name']}")
    print(f"   Job Offers: {best_job['ctc_expected']} LPA in {best_job['city']}")
    print()

# Analyze score distribution
all_scores = recommendations_rdd.flatMap(
    lambda x: [rec[0] for rec in x[2]]
).collect()

if all_scores:
    avg_score = sum(all_scores) / len(all_scores)
    max_score = max(all_scores)
    min_score = min(all_scores)

    print("-"*80)
    print("SCORE STATISTICS:")
    print(f"  Average Recommendation Score: {avg_score:.3f}")
    print(f"  Highest Recommendation Score: {max_score:.3f}")
    print(f"  Lowest Recommendation Score: {min_score:.3f}")

    # Score ranges
    excellent = len([s for s in all_scores if s >= 0.7])
    good = len([s for s in all_scores if 0.5 <= s < 0.7])
    fair = len([s for s in all_scores if 0.3 <= s < 0.5])
    poor = len([s for s in all_scores if s < 0.3])

    print(f"\nSCORE DISTRIBUTION:")
    print(f"  Excellent (≥0.7): {excellent} recommendations ({excellent/len(all_scores)*100:.1f}%)")
    print(f"  Good (0.5-0.7):   {good} recommendations ({good/len(all_scores)*100:.1f}%)")
    print(f"  Fair (0.3-0.5):   {fair} recommendations ({fair/len(all_scores)*100:.1f}%)")
    print(f"  Poor (<0.3):      {poor} recommendations ({poor/len(all_scores)*100:.1f}%)")

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print("\nKEY INSIGHTS:")
print("• Recommendation algorithm uses weighted scoring (Skillset 40%, Experience 30%,")
print("  CTC 20%, City 10%)")
print("• Each employee receives up to 3 job recommendations ranked by fit score")
print("• Higher scores indicate better overall match across all criteria")
print("="*80)


RECOMMENDATION SYSTEM STATISTICS

OVERALL METRICS:
  Total Employees Analyzed: 1000
  Employees with Recommendations: 1000 (100.0%)
  Employees without Recommendations: 0 (0.0%)
  Total Recommendations Generated: 3000
  Average Recommendations per Employee: 3.00
  Total Job Postings Available: 5000

RECOMMENDATION DISTRIBUTION:
  3 recommendations: 1000 employees (100.0%)

--------------------------------------------------------------------------------
TOP 5 EMPLOYEES WITH BEST JOB MATCHES
--------------------------------------------------------------------------------
(Employees whose top recommendation has the highest score)

1. Jose Ramirez - Match Score: 1.000
   Employee: Python, Node.js, React, 6 yrs exp, Pune
   Best Match: DevOps Engineer at Cognizant
   Job Offers: 16.25 LPA in Pune

2. Greg Hines - Match Score: 1.000
   Employee: Angular, Deep Learning, Machine Learning, 1 yrs exp, Pune
   Best Match: Full Stack Developer at Microsoft
   Job Offers: 18.49 LPA in Pune

3. Tim